In [9]:
import numpy as np
import pandas as pd

In [10]:
usecols_all_states = ['z4type', 'effdate']
for i in range(2, 11):
    usecols_all_states.append(f'z4type{i}')
    usecols_all_states.append(f'effdate{i}')
for i in range(1, 11):
    usecols_all_states.append(f'fips{i}')

dtype_all_states = {
    'z4type': 'category',
    'effdate': 'float',
    'fips1': 'object',
}
for i in range(2, 11):
    dtype_all_states[f'z4type{i}'] = 'category'
    dtype_all_states[f'effdate{i}'] = 'float'
    dtype_all_states[f'fips{i}'] = 'object'


df_all_states = pd.read_csv(
    "data/all_states.csv", 
    usecols=usecols_all_states,
    dtype=dtype_all_states
)

In [11]:
list_dict_col_names = [
    {
        'z4type': 'z4type',
        'effdate': 'effdate',
        'fips1': 'fips',
    }
]
for i in range(2, 11):
    list_dict_col_names.append(
        {
            f'z4type{i}': 'z4type',
            f'effdate{i}': 'effdate',
            f'fips{i}': 'fips',
        }
    )


list_df_all_areas = [
    df_all_states[list_dict_col_names[i].keys()].rename(
        columns=list_dict_col_names[i]
    ) for i in range(10)
]

df_all_areas = (
    pd.concat(list_df_all_areas)
        .sort_values('effdate', kind='stable')
        .sort_index(kind='stable')
)

In [40]:
z4types = ('H', 'S', 'empty')

df_all_areas_dropna = df_all_areas.dropna(subset='effdate')

bi_all_dropped = ~(
    df_all_areas_dropna['z4type'].isin(z4types)
        .groupby([df_all_areas_dropna.index, df_all_areas_dropna['effdate']])
        .transform('any')
)

df_all_areas_dropna.loc[bi_all_dropped, 'z4type'] = 'empty'
df_all_areas_dropna.loc[bi_all_dropped, 'fips'] = ""

df_filtered_areas = df_all_areas_dropna[
    df_all_areas_dropna['z4type'].isin(z4types)
]
df_filtered_areas = (
    df_filtered_areas.groupby([df_filtered_areas.index, 'effdate']).first()
)
df_filtered_areas = df_filtered_areas.reset_index('effdate')

In [44]:
df_all_moves = df_filtered_areas[['effdate', 'fips']]
df_all_moves = df_all_moves.rename(columns={'fips': 'destfips'})
df_all_moves['origfips'] = (
    df_all_moves.groupby(df_all_moves.index)['destfips']
    .shift(fill_value="first record")
)